In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter import filedialog as fd
import os
import math

In [ ]:
#User defined constants for the program
crystalline_layers = 2
rdf_cutoff = 8.0 #angstroms
numberbins = 20
x_boundary = "periodic"
y_boundary = "periodic"
z_boundary = "fixed"

In [ ]:
root = Tk()
file = fd.askopenfilename(title = "Choose the a-Si/c-Si interface xyz file")
root.withdraw()

f = open(file, 'r')
iterator = 0
nAtoms = 0
xSize = 0
ySize = 0
zSize = 0
atomicInformationDict = {}
atomicLocations = []
for line in f:
    if iterator == 3:
        nAtoms = float(line)
    if iterator == 5:
        newline = line.split()
        xSize = float(newline[1]) - float(newline[0])
    if iterator == 6:
        newline = line.split()
        ySize = float(newline[1]) - float(newline[0])
    if iterator == 7:
        newline = line.split()
        xSize = float(newline[1]) - float(newline[0])
    elif iterator >= 9:
        newline = line.split()
        floatline = [float(i) for i in newline]
        atomicInformationDict[floatline[0]] = [floatline[1], floatline[2:]]
        atomicLocations.append(floatline[2:])
        
atomicArray = np.array(atomicLocations)
distances = []
#calculate the distance between every atomic pair, if all non-periodic boundary conditions
#for i in range(atomicArray.shape[0] - 1):
#    distances.extend(np.linalg.norm(atomicArray[i+1:,:] - atomicArray[i,:], axis=1))
for n, i in enumerate(atomicArray[:-1]):
    for j in atomicArray[n+1:]:
        if x_boundary == "periodic":
            xDistance = min(abs(i[0]-j[0]),abs(min(i[0],j[0]) - max(i[0], j[0]) + xSize))
        else:
            xDistance = abs(i[0] - j[0])
            
        if y_boundary == "periodic":
            yDistance = min(abs(i[0]-j[0]),abs(min(i[0],j[0]) - max(i[0], j[0]) + ySize))
        else:
            yDistance = abs(i[0] - j[0])
            
        if z_boundary == "periodic":
            zDistance = min(abs(i[0]-j[0]),abs(min(i[0],j[0]) - max(i[0], j[0]) + zSize))
        else:
            zDistance = abs(i[0] - j[0])
        

#apply cutoff and make new array
distances = [i for i in distances if i<rdf_cutoff]    
distanceArray = np.array(distances)

#create histogram bins
count, bins = np.histogram(distanceArray, numberbins)
binwidth = bins[1]-bins[0]
centers = [(i + bins[n])/2 for n, i in enumerate(bins[1:])]

#normalize by the ideal gas
numberDensity = nAtoms/(xSize*ySize*zSize)
normalizedBins = bins/distanceArray.size #first normalize by the total number of particles
for i in range(bins.size):
    idealGasRDF = numberDensity*4*math.pi*centers[i]**2*binwidth
    normalizedBins[i] = normalizedBins[i]/idealGasRDF

In [29]:
line = [[1,1,1],[2,2,2],[3,3,3],[4,4,4],[5,5,5]]
array = np.array(line)
print(array[-1:])
for i in range(array.shape[0]-1):
    print(np.linalg.norm(array[i+1:,:] - array[i,:], axis=1))

[[5 5 5]]
[1.73205081 3.46410162 5.19615242 6.92820323]
[1.73205081 3.46410162 5.19615242]
[1.73205081 3.46410162]
[1.73205081]
